In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
df_series_cfp = pd.read_json('eventseries_wikicfp.json')

In [3]:
df_series_cfp.columns

Index(['dblpSeriesId', 'deleted', 'seriesId', 'title', 'url', 'wikiCfpId',
       'source', 'eventSeriesId', 'acronym'],
      dtype='object')

In [4]:
df_series_cfp[df_series_cfp['dblpSeriesId'].notna()]

,dblpSeriesId,deleted,seriesId,title,url,wikiCfpId,source,eventSeriesId,acronym
0,conf/a2cwic,0,1,A2CWiC: Amrita ACM-W Celebration on Women in C...,http://www.wikicfp.com/cfp/program?id=1,1,wikicfp,1,A2CWiC
1,conf/birthday,0,2,A3L: Algorithmic Algebra and Logic,http://www.wikicfp.com/cfp/program?id=2,2,wikicfp,2,A3L
2,conf/aaai,0,3,AAAI: National Conference on Artificial Intell...,http://www.wikicfp.com/cfp/program?id=3,3,wikicfp,3,AAAI
3,conf/aaa-idea,0,4,AAA-IDEA: Advanced Architectures and Algorithm...,http://www.wikicfp.com/cfp/program?id=4,4,wikicfp,4,AAA-IDEA
4,conf/aacc,0,5,AACC: Asian Applied Computing Conference,http://www.wikicfp.com/cfp/program?id=5,5,wikicfp,5,AACC
...,...,...,...,...,...,...,...,...,...
6014,journals/www,0,6042,World Wide Web,http://www.wikicfp.com/cfp/program?id=6042,6042,wikicfp,6042,None
6015,journals/wwwj,0,6043,World Wide Web Journal,http://www.wikicfp.com/cfp/program?id=6043,6043,wikicfp,6043,None
6016,journals/jwscg,0,6044,WSCG (Journal of ...),http://www.wikicfp.com/cfp/program?id=6044,6044,wikicfp,6044,None
6017,journals/jzusc,0,6045,Zhejiang University - Science C (Journal of ...),http://www.wikicfp.com/cfp/program?id=6045,6045,wikicfp,6045,None


In [5]:
df_event_wikidata = pd.read_json('event_wikidata_with_Time_Acronym_Title.json')

In [232]:
df_event_wikidata.columns

Index(['event', 'eventLabel', 'title', 'acronym', 'startTime', 'endTime',
       'country', 'volumeNumber', 'ceurwsUrl', 'ordinal', 'ceurSptUrl',
       'ceurWsTitle', 'dblpEventId', 'officialWebsite', 'colocatedWith',
       'wikiCfpId', 'wikicfp_seriesID', 'wikicfp_seriesLink',
       'same_time_cfpId', 'same_time_cfpTitle', 'prob_same_cfpId',
       'prob_same_cfpTitle', 'prob_cfpId_LT', 'prob_cfpTitle_LT',
       'prob_cfpId_A', 'prob_cfpTitle_A'],
      dtype='object')

In [7]:
df_event_wikicfp = pd.read_json('event_cfp_with_Time_Acronym_Title.json')

In [231]:
df_event_wikicfp.columns

Index(['Submission_Deadline', 'acronym', 'deleted', 'endDate', 'eventId',
       'eventType', 'locality', 'series', 'seriesId', 'startDate', 'title',
       'wikiCfpId', 'year', 'source', 'url', 'ordinal', 'city',
       'cityWikidataid', 'region', 'regionIso', 'regionWikidataid', 'country',
       'countryIso', 'countryWikidataid', 'lookupAcronym', 'Notification_Due',
       'Final_Version_Due'],
      dtype='object')

wikidataEvent and wikicfpEvent matching with start-end-time(wikidata) and start-end-date(wikicfp)

In [177]:
ll = []
for index, row in df_event_wikidata.iterrows():
    
    df1 = df_event_wikicfp[df_event_wikicfp['startDate']==df_event_wikidata.loc[index,'startTime']]['eventId']
    df2 = df_event_wikicfp[df_event_wikicfp['endDate']==df_event_wikidata.loc[index,'endTime']]['eventId']

    st_list = df1.tolist()
    et_list = df2.tolist()
    #print(st_list)
    #print(et_list)
    same = list(set(st_list) & set(et_list))
    if len(same) == 0:
        ll.append(None)
    else:
        ll.append(same)
    #print("same:",same)
    #df_event_wikidata.loc[index,'same_time_cfpId'] = same
df_event_wikidata['same_time_cfpId'] = ll

In [192]:
titles_cfp=[]
for index, row in df_event_wikidata[df_event_wikidata.same_time_cfpId.notna()].iterrows():
    t_list = []
    for item in row['same_time_cfpId']:
        p_matched_titles = df_event_wikicfp[df_event_wikicfp['eventId']==item]['title']
        #if len(p_matched_titles.values)>1:
        print(type(p_matched_titles.values[0]))
        t_list.append(p_matched_titles.values[0])
    df_event_wikidata.at[index,'same_time_cfpTitle'] = t_list

In [182]:
df_event_wikidata

,event,eventLabel,title,acronym,startTime,endTime,country,volumeNumber,ceurwsUrl,ordinal,...,wikicfp_seriesID,wikicfp_seriesLink,same_time_cfpId,same_time_cfpTitle,prob_same_cfpId,prob_same_cfpTitle,prob_cfpId_LT,prob_cfpTitle_LT,prob_cfpId_A,prob_cfpTitle_A
0,http://www.wikidata.org/entity/Q118698033,Second International Workshop on Linked Data-d...,The 6th edition of the Language Resources and ...,D2R2 2023,2023-05-28,2023-05-28,http://www.wikidata.org/entity/Q41,3401,http://ceur-ws.org/Vol-3401/,2.0,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://www.wikidata.org/entity/Q118425155,International Symposium on Securing NextGenera...,The 18th Meeting of Computational Linguistics ...,SNSFAIT 2023,2023-05-25,2023-05-25,http://www.wikidata.org/entity/Q668,3390,http://ceur-ws.org/Vol-3390/,NaN,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,http://www.wikidata.org/entity/Q118424989,Technology-Enhanced Learning in Laboratories W...,The 22nd International Conference on Computati...,TELL 2023,2023-04-26,2023-04-26,http://www.wikidata.org/entity/Q20,3393,http://ceur-ws.org/Vol-3393/,NaN,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,http://www.wikidata.org/entity/Q118424977,"Workshop on Intervening, Teaming, Delegating",International Joint Conference on Natural Lang...,AutomationXP 2023,2023-04-23,2023-04-23,http://www.wikidata.org/entity/Q183,3394,http://ceur-ws.org/Vol-3394/,NaN,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,http://www.wikidata.org/entity/Q118033825,"REFSQ-2023 Workshops, Doctoral Symposium, Post...",2nd International Conference on Language and A...,REFSQ-JP 2023,2023-04-17,2023-04-20,http://www.wikidata.org/entity/Q29,3378,http://ceur-ws.org/Vol-3378/,NaN,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899,http://www.wikidata.org/entity/Q113673797,Modellierung '98,GeoSensor Networks 2011,Modellierung-97,1998-03-11,1998-03-13,http://www.wikidata.org/entity/Q183,9,http://ceur-ws.org/Vol-9/,NaN,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2900,http://www.wikidata.org/entity/Q113673795,Bildverarbeitung für die Medizin 1996,"International Conference on Control, Automatio...",WMBV'96,1996-11-08,1996-11-09,http://www.wikidata.org/entity/Q183,6,http://ceur-ws.org/Vol-6/,NaN,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2901,http://www.wikidata.org/entity/Q113673796,Workshop on Networked Information Retrieval,International Conference on Advances in Social...,NIR 1996,1996-08-22,1996-08-22,http://www.wikidata.org/entity/Q39,7,http://ceur-ws.org/Vol-7/,NaN,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2902,http://www.wikidata.org/entity/Q113673791,Requirements Engineering für Informationssysteme,2011 IEEE Asia-Pacific Services Computing Conf...,EMISA'95,1995-10-12,1995-10-13,http://www.wikidata.org/entity/Q183,3,http://ceur-ws.org/Vol-3/,NaN,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [187]:
df_event_wikidata[df_event_wikidata.same_time_cfpId.notna()]

,event,eventLabel,title,acronym,startTime,endTime,country,volumeNumber,ceurwsUrl,ordinal,...,wikicfp_seriesID,wikicfp_seriesLink,same_time_cfpId,same_time_cfpTitle,prob_same_cfpId,prob_same_cfpTitle,prob_cfpId_LT,prob_cfpTitle_LT,prob_cfpId_A,prob_cfpTitle_A
61,http://www.wikidata.org/entity/Q116797010,SemREC 2022 and SMART 2022,International Conference on Image and Signal P...,SemREC-SMART 2022,2022-10-24,2022-10-27,http://www.wikidata.org/entity/Q148,3337,http://ceur-ws.org/Vol-3337/,NaN,...,NaN,NaN,[152609],[23th International Conference on Formal Engin...,NaN,NaN,NaN,NaN,NaN,None
78,http://www.wikidata.org/entity/Q116156391,Workshop on Deep Learning for Search and Recom...,32nd Annual IEEE International Computer Softwa...,DL4SR 2022,2022-10-17,2022-10-21,http://www.wikidata.org/entity/Q30,3317,http://ceur-ws.org/Vol-3317/,NaN,...,NaN,NaN,[148821],[The 15th Latin American Theoretical Informati...,NaN,NaN,NaN,NaN,NaN,None
79,http://www.wikidata.org/entity/Q116156388,CIKM 2022 Workshops,The 40th ACM Symposium on Theory of Computing,CIKM-WS 2022,2022-10-17,2022-10-21,http://www.wikidata.org/entity/Q30,3318,http://ceur-ws.org/Vol-3318/,NaN,...,NaN,NaN,[148821],[The 15th Latin American Theoretical Informati...,NaN,NaN,NaN,NaN,NaN,None
84,http://www.wikidata.org/entity/Q114404250,3rd Human-Like Computing Workshop (HLC 2022),International Conference on Compiler Construction,HLC 2022,2022-09-28,2022-09-30,http://www.wikidata.org/entity/Q145,3227,http://ceur-ws.org/Vol-3227/,3.0,...,NaN,NaN,[152890],[14th International Conference on Computationa...,NaN,NaN,NaN,NaN,NaN,None
85,http://www.wikidata.org/entity/Q115250597,Joint Conference of the 31st International Wor...,Human-Robot Interaction,IWSM-MENSURA 2022,2022-09-28,2022-09-30,http://www.wikidata.org/entity/Q43,3272,http://ceur-ws.org/Vol-3272/,NaN,...,NaN,NaN,[152890],[14th International Conference on Computationa...,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2645,http://www.wikidata.org/entity/Q113673271,LMAP/074th Latin Meeting in Analytic Philosophy,6th International Symposium on Visual Computing,LMAP '07,2007-09-20,2007-09-22,http://www.wikidata.org/entity/Q38,278,http://ceur-ws.org/Vol-278/,NaN,...,NaN,NaN,[1172],[8th Hellenic European Research on Computer Ma...,NaN,NaN,NaN,NaN,NaN,NaN
2646,http://www.wikidata.org/entity/Q113649736,CLEF 2007 Workshop,3rd International Conference on Computational ...,CLEF 2007,2007-09-19,2007-09-21,http://www.wikidata.org/entity/Q28,1173,http://ceur-ws.org/Vol-1173/,NaN,...,NaN,NaN,"[1209, 1202, 1228]",[Second International Conference on Digital In...,NaN,NaN,NaN,NaN,NaN,NaN
2647,http://www.wikidata.org/entity/Q113672900,SPDECE '07 IV Simposio Pluridisciplinar sobre ...,3rd ACM Workshop on Many-Task Computing on Gri...,WFM '07,2007-09-19,2007-09-21,http://www.wikidata.org/entity/Q29,318,http://ceur-ws.org/Vol-318/,NaN,...,NaN,NaN,"[1209, 1202, 1228]",[Second International Conference on Digital In...,NaN,NaN,NaN,NaN,NaN,NaN
2657,http://www.wikidata.org/entity/Q113673275,EC-TEL 2007 Posters 2nd European Conference on...,Third NASA Formal Methods Symposium,WFM '07,2007-09-17,2007-09-20,http://www.wikidata.org/entity/Q41,280,http://ceur-ws.org/Vol-280/,2.0,...,NaN,NaN,[1243],[5th International Conference on Service Orien...,NaN,NaN,NaN,NaN,NaN,NaN


wikidataEvent and wikicfpEvent matching with eventLabel(wikidata) and title(wikicfp)

In [141]:
vectorizer = TfidfVectorizer()

for index, row in df_event_wikidata[df_event_wikidata.wikiCfpId.notna() & df_event_wikidata.same_time_cfpTitle.notna()].iterrows():
    id_list = []
    t_list = []
    count = 0

    for item in row['same_time_cfpTitle']:


        #p_matched_titles = p_matched_titles.astype(str)

        vectors_title = vectorizer.fit_transform([row['eventLabel'], item])

    # Calculate cosine similarity between two titles
        similarity = cosine_similarity(vectors_title[0], vectors_title[1])

        if similarity > 0.6:
            print("wikidataTitle:",row['eventLabel'])
            print("wikicfpTitle:", item)
            print("similarity:", similarity)
            id_list.append(int(row['same_time_cfpId'][count]))
            t_list.append(item)
            #t_list = p_matched_titles.tolist()
        count+=1
    if len(id_list) > 0 and len(t_list) > 0 :
        df_event_wikidata.at[index,'prob_cfpId_LT'] = id_list
        df_event_wikidata.at[index,'prob_cfpTitle_LT'] = t_list
    else:
        df_event_wikidata.at[index,'prob_cfpId_LT'] = np.nan
        df_event_wikidata.at[index,'prob_cfpTitle_LT'] = np.nan


wikidataTitle: Workshop Proceedings of the 30th International Conferece on Case-Based Reasoning co-located with the 30th International Conference on Case-Based Reasoning (ICCBR 2022)
wikicfpTitle: International Conference on Case-Based Reasoning
similarity: [[0.60785182]]
wikidataTitle: 34th International Conference on Advanced Information Systems Engineering (CAiSE 2022)
wikicfpTitle: 34th International Conference on Advanced Information Systems Engineering
similarity: [[0.81818021]]
wikidataTitle: 17th International Conference on Persuasive Technology
wikicfpTitle: THE 17TH INTERNATIONAL CONFERENCE ON PERSUASIVE TECHNOLOGIES CONFERENCE
similarity: [[0.65716251]]
wikidataTitle: 18th Italian Research Conference on Digital Libraries
wikicfpTitle: 18th Italian Research Conference on Digital Libraries (IRCDL 2022)
similarity: [[0.799519]]
wikidataTitle: WILF 2021, the 13th International Workshop on Fuzzy Logic and Applications (WILF 2021)
wikicfpTitle: [CfP] Workshop WILF 2021 (13th Inter

wikidataEvent and wikicfpEvent match with Ancronym(wikidata) and Ancronym(wikicfp)

In [190]:
df_event_wikidata['prob_cfpId_A'] = None
df_event_wikidata['prob_cfpTitle_A'] = None
for index, row in df_event_wikidata[df_event_wikidata.same_time_cfpId.notna()].iterrows():
    id_list = []
    t_list = []


    count = 0
    for item in row['same_time_cfpId']:
        p_matched_acronym = df_event_wikicfp[df_event_wikicfp['eventId']==item]['acronym'].values[0]
        #print(p_matched_acronym)
        #p_matched_titles = p_matched_titles.astype(str)

        vectors_title = vectorizer.fit_transform([row['acronym'], p_matched_acronym])

    # Calculate cosine similarity between two titles
        similarity = cosine_similarity(vectors_title[0], vectors_title[1])

        if similarity > 0.5:
            print("wikidata acronym:",row['acronym'])
            print("wikicfp acronym:", p_matched_acronym)
            print("similarity:", similarity)
            id_list.append(item)
            t_list.append(row['same_time_cfpTitle'][count])
            #t_list = p_matched_titles.tolist()
        count+=1
    print(index)
    if len(id_list) > 0 and len(t_list) > 0 :
        #l1.append(id_list)
        #l2.append(t_list)
        df_event_wikidata.at[index,'prob_cfpId_A'] = id_list[:len(id_list)]
        df_event_wikidata.at[index,'prob_cfpTitle_A'] = t_list[:len(t_list)]
    else:
        #l1.append(None)
        #l2.append(None)
        df_event_wikidata.at[index,'prob_cfpId_A'] = None
        df_event_wikidata.at[index,'prob_cfpTitle_A'] = None



61
78
79
84
85
97
104
wikidata acronym: ICCBR-WS 2022
wikicfp acronym: ICCBR 2022
similarity: [[0.70929727]]
112
115
125
126
150
151
152
159
162
165
167
wikidata acronym: CAiSE-DC 2022
wikicfp acronym: CAiSE 2022
similarity: [[0.70929727]]
172
175
wikidata acronym: RCIS-WS&RP 2022
wikicfp acronym: RCIS 2022
similarity: [[0.57973867]]
180
181
184
wikidata acronym: PERSUASIVE-ADJ 2022
wikicfp acronym: PERSUASIVE  2022
similarity: [[0.70929727]]
187
201
202
204
206
208
211
213
wikidata acronym: IRCDL 2022
wikicfp acronym: IRCDL 2022
similarity: [[1.]]
214
wikidata acronym: WILF 2021
wikicfp acronym: WILF 2021
similarity: [[1.]]
221
227
228
229
235
236
237
238
239
250
252
261
263
266
267
268
271
wikidata acronym: ISWC-Posters-Demos-Industry 2021
wikicfp acronym: ISWC 2021
similarity: [[0.50232878]]
278
284
wikidata acronym: ER-Demos-Posters 2021
wikicfp acronym: ER 2021
similarity: [[0.57973867]]
285
287
300
303
305
306
wikidata acronym: RECSYS IN HR 2021
wikicfp acronym: RecSys 2021
simil

In [257]:
df_event_wikidata[df_event_wikidata.prob_cfpId_A.notna() & df_event_wikidata.prob_cfpTitle_A.notna()]

,event,eventLabel,title,acronym,startTime,endTime,country,volumeNumber,ceurwsUrl,ordinal,...,wikicfp_seriesID,wikicfp_seriesLink,same_time_cfpId,same_time_cfpTitle,prob_same_cfpId,prob_same_cfpTitle,prob_cfpId_LT,prob_cfpTitle_LT,prob_cfpId_A,prob_cfpTitle_A
112,http://www.wikidata.org/entity/Q118314674,Workshop Proceedings of the 30th International...,Workshop Proceedings of the 30th International...,ICCBR-WS 2022,2022-09-12,2022-09-15,http://www.wikidata.org/entity/Q142,3389,http://ceur-ws.org/Vol-3389/,NaN,...,1299.0,NaN,"[148121, 153351]",[International Conference on Case-Based Reason...,148121.0,[International Conference on Case-Based Reason...,148121.0,[International Conference on Case-Based Reason...,[148121],[International Conference on Case-Based Reason...
172,http://www.wikidata.org/entity/Q113576239,34th International Conference on Advanced Info...,34th International Conference on Advanced Info...,CAiSE-DC 2022,2022-06-06,2022-06-10,http://www.wikidata.org/entity/Q31,3139,http://ceur-ws.org/Vol-3139/,34.0,...,347.0,NaN,"[150915, 140169, 140170, 140171, 150506, 145678]",[34th International Conference on Advanced Inf...,150915.0,[34th International Conference on Advanced Inf...,150915.0,[34th International Conference on Advanced Inf...,[150915],[34th International Conference on Advanced Inf...
180,http://www.wikidata.org/entity/Q113576115,RCIS 2022 Workshops and Research Projects Track,RCIS 2022 Workshops and Research Projects Track,RCIS-WS&RP 2022,2022-05-17,2022-05-20,http://www.wikidata.org/entity/Q29,3144,http://ceur-ws.org/Vol-3144/,NaN,...,NaN,NaN,"[147090, 147459]","[Research Challenges in Information Science, 3...",NaN,NaN,NaN,NaN,[147090],[Research Challenges in Information Science]
187,http://www.wikidata.org/entity/Q113576032,17th International Conference on Persuasive Te...,17th International Conference on Persuasive Te...,PERSUASIVE-ADJ 2022,2022-03-29,2022-03-31,http://www.wikidata.org/entity/Q846,3153,http://ceur-ws.org/Vol-3153/,17.0,...,2343.0,NaN,[139272],[THE 17TH INTERNATIONAL CONFERENCE ON PERSUASI...,139272.0,[THE 17TH INTERNATIONAL CONFERENCE ON PERSUASI...,139272.0,[THE 17TH INTERNATIONAL CONFERENCE ON PERSUASI...,[139272],[THE 17TH INTERNATIONAL CONFERENCE ON PERSUASI...
214,http://www.wikidata.org/entity/Q113576021,18th Italian Research Conference on Digital Li...,18th Italian Research Conference on Digital Li...,IRCDL 2022,2022-02-24,2022-02-25,http://www.wikidata.org/entity/Q38,3160,http://ceur-ws.org/Vol-3160/,18.0,...,1655.0,NaN,[147701],[18th Italian Research Conference on Digital L...,147701.0,[18th Italian Research Conference on Digital L...,147701.0,[18th Italian Research Conference on Digital L...,[147701],[18th Italian Research Conference on Digital L...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2504,http://www.wikidata.org/entity/Q113672423,The Sixth Spring Young Researchers' Colloquium...,The Sixth Spring Young Researchers' Colloquium...,SYRCoDIS 2009,2009-05-28,2009-05-29,http://www.wikidata.org/entity/Q159,454,http://ceur-ws.org/Vol-454/,6.0,...,2807.0,NaN,[3942],[The Sixth Spring Young Researchers Colloquium...,3942.0,[The Sixth Spring Young Researchers Colloquium...,3942.0,[The Sixth Spring Young Researchers Colloquium...,[3942],[The Sixth Spring Young Researchers Colloquium...
2507,http://www.wikidata.org/entity/Q113672446,AIAI 2009,AIAI 2009,AIAI 2009,2009-04-23,2009-04-25,http://www.wikidata.org/entity/Q41,475,http://ceur-ws.org/Vol-475/,NaN,...,NaN,NaN,[3088],[5th IFIP International Conference on Artifici...,NaN,NaN,NaN,NaN,[3088],[5th IFIP International Conference on Artifici...
2549,http://www.wikidata.org/entity/Q113672656,I-USED 2008 First International Workshop on th...,I-USED 2008 First International Workshop on th...,I-USED 2008,2008-09-24,2008-09-24,http://www.wikidata.org/entity/Q38,407,http://ceur-ws.org/Vol-407/,1.0,...,1802.0,NaN,[3193],[International Workshop on the Interplay betwe...,3193.0,[International Workshop

In [222]:
df_es = pd.DataFrame(pd.read_json('event_series.json').results.bindings)

df_es

,series,title,acronym,instanceOf,dblpVenueId,officialWebsite,wikiCfpSeriesId
0,"{'type': 'uri', 'value': 'http://www.wikidata....","{'xml:lang': 'en', 'type': 'literal', 'value':...","{'xml:lang': 'en', 'type': 'literal', 'value':...","{'type': 'literal', 'value': 'http://www.wikid...","{'type': 'literal', 'value': 'conf/eurohcir'}",NaN,NaN
1,"{'type': 'uri', 'value': 'http://www.wikidata....","{'xml:lang': 'en', 'type': 'literal', 'value':...","{'xml:lang': 'en', 'type': 'literal', 'value':...","{'type': 'literal', 'value': 'http://www.wikid...","{'type': 'literal', 'value': 'conf/nesy'}","{'type': 'uri', 'value': 'http://www.neural-sy...",NaN
2,"{'type': 'uri', 'value': 'http://www.wikidata....","{'xml:lang': 'en', 'type': 'literal', 'value':...","{'xml:lang': 'en', 'type': 'literal', 'value':...","{'type': 'literal', 'value': 'http://www.wikid...","{'type': 'literal', 'value': 'conf/colins'}",NaN,NaN
3,"{'type': 'uri', 'value': 'http://www.wikidata....","{'xml:lang': 'en', 'type': 'literal', 'value':...","{'xml:lang': 'en', 'type': 'literal', 'value':...","{'type': 'literal', 'value': 'http://www.wikid...","{'type': 'literal', 'value': 'conf/woa'}",NaN,NaN
4,"{'type': 'uri', 'value': 'http://www.wikidata....","{'xml:lang': 'en', 'type': 'literal', 'value':...","{'xml:lang': 'en', 'type': 'literal', 'value':...","{'type': 'literal', 'value': 'http://www.wikid...","{'type': 'literal', 'value': 'conf/immoa'}",NaN,NaN
...,...,...,...,...,...,...,...
128,"{'type': 'uri', 'value': 'http://www.wikidata....","{'xml:lang': 'en', 'type': 'literal', 'value':...","{'xml:lang': 'en', 'type': 'literal', 'value':...","{'type': 'literal', 'value': 'http://www.wikid...","{'type': 'literal', 'value': 'conf/cilc'}",NaN,"{'type': 'literal', 'value': '463'}"
129,"{'type': 'uri', 'value': 'http://www.wikidata....","{'xml:lang': 'en', 'type': 'literal', 'value':...","{'xml:lang': 'en', 'type': 'literal', 'value':...","{'type': 'literal', 'value': 'http://www.wikid...","{'type': 'literal', 'value': 'conf/benevol'}",NaN,NaN
130,"{'type': 'uri', 'value': 'http://www.wikidata....","{'xml:lang': 'en', 'type': 'literal', 'value':...","{'xml:lang': 'en', 'type': 'literal', 'value':...","{'type': 'literal', 'value': 'http://www.wikid...","{'type': 'literal', 'value': 'conf/syrcodis'}",NaN,NaN
131,"{'type': 'uri', 'value': 'http://www.wikidata....",NaN,"{'xml:lang': 'en', 'type': 'literal', 'value':...","{'type': 'literal', 'value': 'http://www.wikid...",NaN,NaN,NaN


In [225]:
#df_es['series'] = df_es['series'].apply(lambda x: x['value'])
#df_es[df_es.title.notna()]
#df_es['title'] = df_es['title']
df_es['acronym'] = df_es['acronym'].apply(extract_value)
df_es['instanceOf'] = df_es['instanceOf'].apply(extract_value)
df_es['dblpVenueId'] = df_es['dblpVenueId'].apply(extract_value)
df_es['officialWebsite'] = df_es['officialWebsite'].apply(extract_value)

In [219]:
def extract_value(dic):
    if pd.notna(dic):
        return dic.get('value')
    else:
        return dic

In [228]:
df_es.to_json('event_series_std.json', orient='records')

In [236]:
df_es

,series,title,acronym,instanceOf,dblpVenueId,officialWebsite,wikiCfpSeriesId,wikiCfpSeriesTitle
0,http://www.wikidata.org/entity/Q105699531,European Workshop on Human-Computer Interactio...,EuroHCIR,http://www.wikidata.org/entity/Q47258130,conf/eurohcir,NaN,None,None
1,http://www.wikidata.org/entity/Q105696159,International Workshop on Neural-Symbolic Lear...,NeSy,http://www.wikidata.org/entity/Q47258130,conf/nesy,http://www.neural-symbolic.org/,None,None
2,http://www.wikidata.org/entity/Q105699749,International Conference on Computational Ling...,COLINS,http://www.wikidata.org/entity/Q47258130 http:...,conf/colins,NaN,None,None
3,http://www.wikidata.org/entity/Q105699022,Workshop From Objects to Agents,WOA,http://www.wikidata.org/entity/Q47258130 http:...,conf/woa,NaN,None,None
4,http://www.wikidata.org/entity/Q105700233,International Workshop on Information Manageme...,IMMoA,http://www.wikidata.org/entity/Q47258130,conf/immoa,NaN,None,None
...,...,...,...,...,...,...,...,...
128,http://www.wikidata.org/entity/Q105696183,Italian Conference on Computational Logic,CILC,http://www.wikidata.org/entity/Q47258130 http:...,conf/cilc,NaN,None,None
129,http://www.wikidata.org/entity/Q105699329,BElgian-NEtherlands software eVOLution symposium,BENEVOL,http://www.wikidata.org/entity/Q47258130,conf/benevol,NaN,None,None
130,http://www.wikidata.org/entity/Q105696315,Spring Young Researchers Colloquium on Databas...,SYRCoDIS,http://www.wikidata.org/entity/Q47258130,conf/syrcodis,NaN,None,None
131,http://www.wikidata.org/entity/Q113512475,NaN,MK,http://www.wikidata.org/entity/Q47459256,NaN,NaN,None,None


In [230]:
df_event_wikidata[df_event_wikidata.prob_cfpId_A.notna() & df_event_wikidata.prob_cfpTitle_A.notna() & df_event_wikidata.prob_cfpId_LT.notna() & df_event_wikidata.same_time_cfpId.notna()]

,event,eventLabel,title,acronym,startTime,endTime,country,volumeNumber,ceurwsUrl,ordinal,...,wikicfp_seriesID,wikicfp_seriesLink,same_time_cfpId,same_time_cfpTitle,prob_same_cfpId,prob_same_cfpTitle,prob_cfpId_LT,prob_cfpTitle_LT,prob_cfpId_A,prob_cfpTitle_A
112,http://www.wikidata.org/entity/Q118314674,Workshop Proceedings of the 30th International...,Workshop Proceedings of the 30th International...,ICCBR-WS 2022,2022-09-12,2022-09-15,http://www.wikidata.org/entity/Q142,3389,http://ceur-ws.org/Vol-3389/,NaN,...,1299.0,NaN,"[148121, 153351]",[International Conference on Case-Based Reason...,148121.0,[International Conference on Case-Based Reason...,148121.0,[International Conference on Case-Based Reason...,[148121],[International Conference on Case-Based Reason...
172,http://www.wikidata.org/entity/Q113576239,34th International Conference on Advanced Info...,34th International Conference on Advanced Info...,CAiSE-DC 2022,2022-06-06,2022-06-10,http://www.wikidata.org/entity/Q31,3139,http://ceur-ws.org/Vol-3139/,34.0,...,347.0,NaN,"[150915, 140169, 140170, 140171, 150506, 145678]",[34th International Conference on Advanced Inf...,150915.0,[34th International Conference on Advanced Inf...,150915.0,[34th International Conference on Advanced Inf...,[150915],[34th International Conference on Advanced Inf...
187,http://www.wikidata.org/entity/Q113576032,17th International Conference on Persuasive Te...,17th International Conference on Persuasive Te...,PERSUASIVE-ADJ 2022,2022-03-29,2022-03-31,http://www.wikidata.org/entity/Q846,3153,http://ceur-ws.org/Vol-3153/,17.0,...,2343.0,NaN,[139272],[THE 17TH INTERNATIONAL CONFERENCE ON PERSUASI...,139272.0,[THE 17TH INTERNATIONAL CONFERENCE ON PERSUASI...,139272.0,[THE 17TH INTERNATIONAL CONFERENCE ON PERSUASI...,[139272],[THE 17TH INTERNATIONAL CONFERENCE ON PERSUASI...
214,http://www.wikidata.org/entity/Q113576021,18th Italian Research Conference on Digital Li...,18th Italian Research Conference on Digital Li...,IRCDL 2022,2022-02-24,2022-02-25,http://www.wikidata.org/entity/Q38,3160,http://ceur-ws.org/Vol-3160/,18.0,...,1655.0,NaN,[147701],[18th Italian Research Conference on Digital L...,147701.0,[18th Italian Research Conference on Digital L...,147701.0,[18th Italian Research Conference on Digital L...,[147701],[18th Italian Research Conference on Digital L...
221,http://www.wikidata.org/entity/Q113580006,"WILF 2021, the 13th International Workshop on ...","WILF 2021, the 13th International Workshop on ...",WILF 2021,2021-12-20,2021-12-22,http://www.wikidata.org/entity/Q38,3074,http://ceur-ws.org/Vol-3074/,13.0,...,3073.0,NaN,"[145017, 144034, 135269, 143602]",[The 15th International Conference on Algorith...,135269.0,[[CfP] Workshop WILF 2021 (13th International ...,135269.0,[[CfP] Workshop WILF 2021 (13th International ...,[135269],[[CfP] Workshop WILF 2021 (13th International ...
318,http://www.wikidata.org/entity/Q113580156,CLEF 2021 - Conference and Labs of the Evaluat...,CLEF 2021 - Conference and Labs of the Evaluat...,CLEF-WN 2021,2021-09-21,2021-09-24,http://www.wikidata.org/entity/Q218,2936,http://ceur-ws.org/Vol-2936/,NaN,...,494.0,NaN,"[127819, 120806]","[Conference and Labs of the Evaluation Forum, ...",127819.0,[Conference and Labs of the Evaluation Forum],127819.0,[Conference and Labs of the Evaluation Forum],[127819],[Conference and Labs of the Evaluation Forum]
337,http://www.wikidata.org/entity/Q113580039,29th International Conference on Case-Based Re...,29th International Conference on Case-Based Re...,ICCBR-WS 2021,2021-09-13,2021-09-16,http://www.wikidata.org/entity/Q29,3017,http://ceur-ws.org/Vol-3017/,29.0,...,1299.0,NaN,[125128],[29th International Conference on Case-Based R...,125128.0,[29th International Conference on Case-Based R...,125128.0,[29th International Conference on Case-Based R...,[125128],[29th International Conference on Case-Based R...
455,http://www.wikidata.org/entity/Q113581832,13th European Workshop on Services and their C...,13th European Wor

In [234]:
df_event_wikidata.to_json('event_wikidata_matched.json', orient='records')

In [255]:
df_es['wikiCfpSeriesId'] = None
df_es['wikiCfpSeriesTitle'] = None
for index, row in df_es[df_es.dblpVenueId.notna()].iterrows():
    find = df_series_cfp[df_series_cfp['dblpSeriesId']==row['dblpVenueId']]['eventSeriesId'].values

    find2 = df_series_cfp[df_series_cfp['dblpSeriesId']==row['dblpVenueId']]['title'].values

    if len(find) == 0 and len(find2) ==0:

        df_es.at[index,'wikiCfpSeriesId'] = None
        df_es.at[index,'wikiCfpSeriesTitle'] = None
    else:
        df_es.at[index,'wikiCfpSeriesId'] = find
        df_es.at[index,'wikiCfpSeriesTitle'] = find2


In [256]:
df_es[df_es.wikiCfpSeriesId.notna()]


,series,title,acronym,instanceOf,dblpVenueId,officialWebsite,wikiCfpSeriesId,wikiCfpSeriesTitle
3,http://www.wikidata.org/entity/Q105699022,Workshop From Objects to Agents,WOA,http://www.wikidata.org/entity/Q47258130 http:...,conf/woa,NaN,[3119],[WOA: Workshop From Objects to Agents ]
12,http://www.wikidata.org/entity/Q105698459,International Workshop on Modular Ontologies,WoMO,http://www.wikidata.org/entity/Q47258130,conf/womo,NaN,[3128],[WoMO: Workshop on Modular Ontologies ]
13,http://www.wikidata.org/entity/Q105698581,Design and Management of Data Warehouses,DMDW,http://www.wikidata.org/entity/Q47258130 http:...,conf/dmdw,NaN,[735],[DMDW: Design and Management of Data Warehouses ]
17,http://www.wikidata.org/entity/Q105698439,International Conference on Biomedical Ontology,ICBO,http://www.wikidata.org/entity/Q47258130 http:...,conf/icbo,NaN,[1293],[ICBO: International Conference on Biomedical ...
21,http://www.wikidata.org/entity/Q105700260,International Workshop on Online Dispute Resol...,ODR,http://www.wikidata.org/entity/Q47258130,conf/odr,NaN,[2253],[ODR: Online Dispute Resolution ]
26,http://www.wikidata.org/entity/Q105700408,Latin-American Workshop on Non-Monotonic Reaso...,LA-NMR,http://www.wikidata.org/entity/Q47258130 http:...,conf/lanmr,NaN,[1934],[LA-NMR: Latin-American Workshop on Non-Monoto...
28,http://www.wikidata.org/entity/Q105698448,International Conference on Semantic Systems,SEMANTICS,http://www.wikidata.org/entity/Q47258130,conf/i-semantics,NaN,[1705],[I-SEMANTICS: International Conference on Sema...
32,http://www.wikidata.org/entity/Q105695719,International Conference of the Italian Associ...,AI*IA,http://www.wikidata.org/entity/Q47258130 http:...,conf/aiia,NaN,[106],[AI*IA: Congress of the Italian Association fo...
34,http://www.wikidata.org/entity/Q105696273,Quality of Information and Communications Tech...,QUATIC,http://www.wikidata.org/entity/Q47258130,conf/quatic,NaN,[2453],[QUATIC: Quality of Information and Communicat...
39,http://www.wikidata.org/entity/Q105700662,Workshop on Advanced Learning Technologies for...,WALTD,http://www.wikidata.org/entity/Q47258130,conf/waltd,NaN,[3003],[WALTD: Workshop on Advanced Learning Technolo...


In [251]:
df_es

,series,title,acronym,instanceOf,dblpVenueId,officialWebsite,wikiCfpSeriesId,wikiCfpSeriesTitle
0,http://www.wikidata.org/entity/Q105699531,European Workshop on Human-Computer Interactio...,EuroHCIR,http://www.wikidata.org/entity/Q47258130,conf/eurohcir,NaN,[],[]
1,http://www.wikidata.org/entity/Q105696159,International Workshop on Neural-Symbolic Lear...,NeSy,http://www.wikidata.org/entity/Q47258130,conf/nesy,http://www.neural-symbolic.org/,[],[]
2,http://www.wikidata.org/entity/Q105699749,International Conference on Computational Ling...,COLINS,http://www.wikidata.org/entity/Q47258130 http:...,conf/colins,NaN,[],[]
3,http://www.wikidata.org/entity/Q105699022,Workshop From Objects to Agents,WOA,http://www.wikidata.org/entity/Q47258130 http:...,conf/woa,NaN,[3119],[WOA: Workshop From Objects to Agents ]
4,http://www.wikidata.org/entity/Q105700233,International Workshop on Information Manageme...,IMMoA,http://www.wikidata.org/entity/Q47258130,conf/immoa,NaN,[],[]
...,...,...,...,...,...,...,...,...
128,http://www.wikidata.org/entity/Q105696183,Italian Conference on Computational Logic,CILC,http://www.wikidata.org/entity/Q47258130 http:...,conf/cilc,NaN,[463],[CILC: Italian Conference on Computational Log...
129,http://www.wikidata.org/entity/Q105699329,BElgian-NEtherlands software eVOLution symposium,BENEVOL,http://www.wikidata.org/entity/Q47258130,conf/benevol,NaN,[],[]
130,http://www.wikidata.org/entity/Q105696315,Spring Young Researchers Colloquium on Databas...,SYRCoDIS,http://www.wikidata.org/entity/Q47258130,conf/syrcodis,NaN,[2807],[SYRCoDIS: Spring Young Researchers Colloquium...
131,http://www.wikidata.org/entity/Q113512475,NaN,MK,http://www.wikidata.org/entity/Q47459256,NaN,NaN,None,None


In [6]:
from eventseries.src.main.dblp.EventClasses import Event, DblpEvent, EventSeries
import pickle
from typing import Optional, Tuple, Union, List

ModuleNotFoundError: No module named 'VenueInformationClasses'

In [12]:
from pathlib import Path

ModuleNotFoundError: No module named 'VenueInformationClasses'

In [11]:
def load_event_series(path: Path = Path('.') / '..' / 'resources' / 'dblp_event_series.pickle') -> List[EventSeries]:
    with open(path, 'rb') as file:
        event_series: List[EventSeries] = pickle.load(file)
        return event_series

ModuleNotFoundError: No module named 'VenueInformationClasses'